In [2]:
import json
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from PyPDF2 import PdfReader
import requests

# File paths
CLIENT_SECRET_FILE = "client_secret.json"  # Replace with your client secret file name
TOKEN_FILE = "token.json"  # File to save user credentials

# Scopes and API endpoint
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]
API_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta2/models/text-bison-001:generateText"

# Function to authenticate and get an access token
def authenticate_and_get_token():
    creds = None
    # Load credentials if they exist
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    # If no credentials or expired, perform authentication
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=8000)  # Ensure port matches redirect URI
        # Save credentials for future use
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())
    return creds.token

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for page in reader.pages:
        extracted_text += page.extract_text()
    return extracted_text

# Function to analyze text using Google Generative AI API
def analyze_text_with_google(text):
    token = authenticate_and_get_token()
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    payload = {
        "prompt": f"Analyze this medical report and provide detailed insights:\n\n{text}",
        "temperature": 0.7,
        "maxOutputTokens": 300
    }
    response = requests.post(API_ENDPOINT, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json().get("candidates", [{}])[0].get("output", "No output received.")
    else:
        return f"Error: {response.status_code}, {response.text}"

# Main function
def main():
    try:
        pdf_path = input("Enter the path to the medical report PDF: ").strip()
        if not os.path.exists(pdf_path):
            print("Error: File not found!")
            return

        # Extract text from the PDF
        print("\nExtracting text from the PDF...")
        extracted_text = extract_text_from_pdf(pdf_path)
        print("\nExtracted Text (First 1000 characters):\n", extracted_text[:1000], "...")

        # Analyze text with Google Generative AI
        print("\nAnalyzing the text with Google Generative AI...\n")
        result = analyze_text_with_google(extracted_text)
        print("\nAnalysis Result:\n", result)

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()



Extracting text from the PDF...

Extracted Text (First 1000 characters):
 SRIKRISHNASCANCENTRE
Opp:BusStand,NIDADAVOLE-534301.W.G.Dt.,A.P.PNDT/Rc.No.223/2017
Dr.D.Haritha,M.D.,(Radiodiagnosis)-Cell:9533223318
Patientname:Mrs.P.ANUSHA
Age/Sex:26Years/Female
VisitNo:1PatientID:2204202003
Referredby:Dr.
M.SOWJANYA
VisitDate:22/04/2020
PelvisReport
RealtimeB-modeUltrasonographyofPelvisdone
Pelvis:TransabdominalandTransvaginalsonographyofthepelvisdone
Uterus:Uterusmeasured5.7X2.4X2.4cms.
Normalinsizeandechotexture.Nofocallesions.
EndometrialCavity:Endometriumisthickenedmeasuirng8mm.
RightOvary:Rightovarymeasured4.1X2.2X2.8cms.(Volume=13.13cc.)
LeftOvary:Leftovarymeasured3.9X2.1X2.7cms.(Volume=11.5cc.)
Bothovariesshowmultiplesmallcystsofuniformsizearrangedperipherally.Fewsmall
developingfolliclesseeninthecentralstroma.
RightAdnexa:Rightadnexaappearednormal
LeftAdnexa:Leftadnexaappearednormal
PouchofDouglas:Normal.
Impression
*Thickenedendometrium(8mm).PatientisUPTnegative.
*Polycysticappear

: 